 ###

<center><h1> $\text{Arrow Debreu Option Pricing Implementation}$ </h1> </center>
<center>$\text{by Arsenii H., Institute of Mathematical Finance}$</center>

### I: Introduction. 
> This notebook implements the Arrow-Debreu [model](https://en.wikipedia.org/wiki/Arrow%E2%80%93Debreu_model) to "fairly" price European options and detect arbitrage opportunities, alongside calculating the risk free profit. The ideas is to consider every possible contingent claim based on [sate prices](https://en.wikipedia.org/wiki/State_prices).

### I.I: Definitions.
* **Option** - The right, but not the obligation to buy or sell a hundred units of the underying security at a predetermined price called "strike", and on a predetermined date called "expiry".
* **Terminal States** - Possible future prices of the underlying asset. Denoted $S = \{S_1, S_2, \ldots, S_n\}$ This is a discrete approximation for what is actually a continuous distribution. However for practical purposes and we choose feasible discrete intervals. (ex. 80, 90, 100, 110).
* **Payoff Vector** - Each option corresponds to a row vector of payoffs across terminal states. These row vectors make up the matrix $F$.
* **State Price** - Also referred to as Arrow Debreu price the present value of recieving 1 dollar iff the underlying price at expiration ends up in that specific terminal state. It reflects the time-value of money, the market consensus about future risk, and no arbitrage (ex. suppose there're only 3 possible terminal states the underlying could end up at: $ S_{i} \in \{80, 90, 100\} $, then the state prices $\vec{\pi} = [\pi_{80}, \pi_{90}, \pi_{100}]$ tells us today's price of recieving 1 dollar in the future is the underlyig ends at that subscript).
* **[Risk-Neutral Probailities](https://www.investopedia.com/terms/r/risk-neutral-probabilities.asp)** - the probabilities of possible future outcomes that have been asjusted for risk, denoted $q_i=\frac{\pi_{i}}{e^{-rT}}=\pi_{i} \cdot e^{rT}$
* **[Complete Market](https://en.wikipedia.org/wiki/Complete_market)** - a market with negligible transaction costs, perfect information and a set price for every asset in every possible state. For practical purposes some of these assumptions may be altered.

### II: General Form.

<center style="font-size: 150%">
$\vec{p} = F\vec{\pi}$
</center>

* $\vec{p}$ - a vector of current call/put prices (all the same maturity). <br>
* ${F}$ - ${m \times n}$ matrix of payoffs with $m$ options and $n$ states. Where each row corresponds to an option and each column corresponds to a possible terminal state of the underlying.
* $\vec{\pi}$ - a vector of state prices.
> Each observed option price is a weighted average of possible future payoffs, weighted by the present value of receiving 1 dollar in that state. The payoff matrix $F$ maps states to payoffs. The vector $\vec{\pi}$ maps states to dollar values today. Then, multiplying them gives the current prices of the options.

### II.I: Solve For $\vec{\pi}$.

<center style = "font-size: 150%">
    $\vec{\pi} = F^{-1}\vec{p}$
</center>

> At this step we invert the payoff matrix (assuming it's invertable) using NumPy, solving for the implied Arrow-Debreu securities prices, in our case options.

### III: Fair Price.

<center style = "font-size: 150%">
    $\alpha = \displaystyle\sum_{i=1}^n{\pi_i} \cdot f(S_{i}) = \langle \vec{\pi} \cdot \vec{f(S_i)} \rangle$
</center>

* $\alpha$ - the fair price of an option. <br>
* $\pi_{i}$ - the Arrow Debreu price for state $i$. <br>
* $f(S_{i})$ - the payoff of terminal price $S$ at state $i$ of option $p$.
* Note: the summation version is equivalent to the dot product version.
* For call options with strike K:
<center>$f_c(S_{i}) = max(0, S_i - K)$ </center>
* For put options with strike K: 
<center>$f_p(S_{i}) = max(0, K - S_i)$</center>

### IV: Probabilites.

<center style = "font-size: 120%">
    $q_i=\pi_{i} \cdot e^{rT} \implies \displaystyle\sum_{i}^n{q_i}=1$ 
</center>

> This works because each $\pi_i$ is what you'd pay today to recieve 1 dollar iff state $i$ occurs according to the market implied likelihood. And since everything is scaled to a dollar that becomes the implied probability when adjusted for the time value of money. And since $q$ is risk-neutral probabilities, its elements must sum to one.

### V: Data.

* <ins>Strike Prices</ins>: $K = \{K_1, K_2, \ldots , K_n\}$. They define the contracts used to construct the system (preferably the same for calls and puts).
* <ins>Call/Put Options Premiums</ins>: $ \vec{p} = [p_1, p_2, \ldots, p_n]$ i.e the current market price of options at each $K_{i}$.
* <ins>Underlying Spot Price</ins>: $S$ to help define reasonable strike prices.
* <ins>Time to Maturity</ins>: $T$ (in years).
* <ins>Risk-Free Rate</ins>: $r$ (T-bill/note/bond yeild).
* <ins>Discounting Factor</ins>: $e^{-rT}$ Continuously compounding discounting factor. Optional for short-term, but necessary for risk-neutral probabilities.